# 유사한 단어 찾기 게임

1. 사전 학습된 모델 또는 적절한 데이터셋을 찾는다
2. 워드 임베딩 모델을 학습시킨다
3. 단어 유사도가 0.8 이상인 A, B를 랜덤 추출한다
4. A, B와 대응되는 C를 추출한다
5. D를 입력받는다

=><br>
A:B = C:D 관계에 대응하는 D를 찾는 게임을 만든다.<br>
ex) A: 산, B: 바다, C: 나무, D: 물

**<출력 예시>**

관계 [ 수긍 : 추락 = 대사관 : ? ]<br>
모델이 예측한 가장 적합한 단어: 잠입<br>
당신의 답변과 모델 예측의 유사도: 0.34<br>
아쉽네요. 더 생각해보세요.

##### 이 게임이 너무 어렵다면?

=> 우선 Semantle(https://semantle.com/ 또는 한국어 버전 꼬맨틀/뉴맨틀) 흉내내서 만들면서 환기해보기~!

In [86]:
# !pip install gensim

In [87]:
# !pip install openpyxl

In [88]:
# !python -m gensim.scripts.word2vec2tensor --input ted_en_w2v --output ted_en_w2v

In [89]:
import pandas as pd
import openpyxl

# 파일 로드
communication_df = pd.read_excel('/Users/woojin/Desktop/SK Networks Family AI Camp_17/SKN-17-Family-AI-Camp/NATURAL_LANGUAGE_PROCESSING/03_word_embedding/한국어_단발성_대화_데이터셋.xlsx')
communication_df

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
0,언니 동생으로 부르는게 맞는 일인가요..??,공포,NaN,NaN,NaN,놀람,5898.0
1,그냥 내 느낌일뿐겠지?,공포,NaN,NaN,NaN,분노,5665.0
2,아직너무초기라서 그런거죠?,공포,NaN,NaN,NaN,슬픔,5267.0
3,유치원버스 사고 낫다던데,공포,NaN,NaN,NaN,중립,4830.0
4,근데 원래이런거맞나요,공포,NaN,NaN,NaN,행복,6037.0
...,...,...,...,...,...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라..,혐오,NaN,NaN,NaN,NaN,NaN
38590,재미가 없으니 망하지,혐오,NaN,NaN,NaN,NaN,NaN
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오,NaN,NaN,NaN,NaN,NaN
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스 ㅉㅉㅉ,혐오,NaN,NaN,NaN,NaN,NaN


In [90]:
# 필요없는 컬럼 제거
communication_df = communication_df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', '공포', 5468], axis=1).reset_index(drop=True)
communication_df.head()

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,공포
1,그냥 내 느낌일뿐겠지?,공포
2,아직너무초기라서 그런거죠?,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포


In [91]:
# 결측치 처리
communication_df.isnull().sum()

Sentence    0
Emotion     0
dtype: int64

In [92]:
communication_df['Sentence'][300:400]

300                                            국회의원이라고?
301                                       불출석했으면서 자백강요?
302                                 비교할사람이랑 비교를 해야지;;;;
303    음.. 마광수 작품은 음란패설이 담긴 책이고, 참을 수 없는 존재의 가벼움은 명작이다?
304                                     증거는..??????????
                             ...                       
395                                     다른 손님들도 보고 있는데?
396                             어리지도 많지도 않은 애매한 나이인것같아요
397                  누가봐도 그 동생이 좋아하는 건데 자기는 아니라고 하더라고요.
398            이번방학에 토스 해보려고 하는데요인강은 제가 잘 할수있을지 모르겠어요;;
399                                      흠 뭐라고 해야 할까여?0
Name: Sentence, Length: 100, dtype: object

In [93]:
# 한글이 아닌 데이터 제거
communication_df['Sentence'] = communication_df['Sentence'].replace(r'[^가-힣\s]', '', regex=True)

In [94]:
from konlpy.tag import Okt
from tqdm import tqdm

okt = Okt()
# ko_stopwords = ["은", "는", "이", "가", "을", "를", "과", "와", "들", "도", "부터", "까지", "에", "나", "너", "그", "걔", "얘"]
# ko_stopwords = set("은 는 이 가 을 를 과 와 들 도 부터 까지 에 나 너 그 걔 얘 보다 때문에 있는 있습니다 아 휴 아이구 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 그리고 비길수 없다 해서는 안된다 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야 된이상 만큼 어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국 의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 네나".split())
def load_stopwords(filepath):
    with open(filepath, 'r', encoding='UTF-8') as f:
        stopwords = [line.strip() for line in f]
    return stopwords

ko_stopwords = load_stopwords('ko_stopwords.txt')

preprocessed_sentences = []

for sentence in tqdm(communication_df['Sentence']):
    tokens = okt.morphs(sentence, stem=True)
    tokens = [nouns for token in tokens if token not in ko_stopwords for nouns in okt.nouns(token) if nouns]
    preprocessed_sentences.append(tokens)
    
preprocessed_sentences

100%|██████████| 38594/38594 [00:48<00:00, 796.73it/s] 


[['언니', '동생'],
 ['그냥', '내', '느낌', '뿐'],
 ['초기', '라서', '거'],
 ['유치원', '버스', '사고', '낫다'],
 ['원래'],
 ['남자친구', '요'],
 ['거', '허리'],
 ['내'],
 ['일주일', '도', '안'],
 ['약', '은', '최대한', '안', '음시'],
 ['구직', '활동', '은', '뭘', '활용'],
 ['분'],
 ['스타일', '더'],
 ['아내', '여자', '술집', '가면', '법', '적', '문제'],
 ['한소리'],
 ['분'],
 ['그냥', '학업', '관련', '된거', '더'],
 ['가격', '은', '반값', '거', '질', '건가', '요'],
 ['마지막', '말씀', '글', '은', '이제', '리다', '의도', '치', '실시간', '베스트', '글', '남친'],
 ['화장품', '회사', '여자', '걱정', '요'],
 ['몸무게', '키', '찌'],
 ['다리'],
 ['안'],
 ['정도'],
 ['보통', '진통', '도', '가나', '요'],
 ['여자', '술취해', '먼저'],
 ['걱정'],
 ['거', '증상'],
 ['동생', '걱정'],
 ['무리'],
 ['애인', '정말', '걱정'],
 ['주', '일전'],
 ['경고', '고민'],
 ['극도', '상태'],
 ['못'],
 ['안검', '하수도', '요'],
 ['그게', '사마귀', '일수'],
 ['이제', '내년', '이면', '살인', '여대생', '요즘', '고민'],
 ['안'],
 ['자제', '방법', '뭐'],
 ['집', '운동', '지난', '주의', '노력', '물', '거품', '정말', '걱정'],
 ['선택', '인지'],
 ['면', '둘째', '마음', '더'],
 ['엄마', '친구', '안'],
 ['내', '구글', '폰', '계정', '동기화', '안', '어쩌'],
 ['더'],
 [],
 [],
 ['깊이',

In [95]:
from gensim.models import Word2Vec

model = Word2Vec(
    sentences=preprocessed_sentences,
    vector_size=200,
    window=8,
    min_count=5,
    sg=1,                # Skip-gram
    negative=10,
    epochs=15,
    workers=4,
    sample=1e-3,
    seed=42
)

model.wv.vectors.shape
model.wv.most_similar('박근혜')

[('퇴진', 0.6920600533485413),
 ('자백', 0.6539689302444458),
 ('대행', 0.6497374773025513),
 ('황교안', 0.6461187601089478),
 ('부결', 0.641380250453949),
 ('닭그네', 0.6411082744598389),
 ('즉각', 0.6403540372848511),
 ('독재자', 0.6385412216186523),
 ('하야', 0.6320891976356506),
 ('구속', 0.630487322807312)]

In [96]:
model.wv.save_word2vec_format('communication_w2v')

In [97]:
from gensim.models import KeyedVectors

load_model = KeyedVectors.load_word2vec_format('communication_w2v')

In [101]:
import random
from numpy.linalg import norm
import numpy as np

def get_word_pair(kv, threshold=0.8, upper=0.98):
    """A,B: 서로 충분히 비슷하지만(>=threshold) 지나치게 동일하지 않게(<upper)."""
    words = list(kv.key_to_index.keys())
    while True:
        A, B = random.sample(words, 2)
        s = kv.similarity(A, B)
        if threshold <= s <= upper:
            return A, B, s

def pick_C_similar(kv, A=None, B=None, base="A", topk=60, min_sim=0.45, max_sim=0.85):
    """
    C: A(또는 B)와 적당히 비슷한 단어에서 고름.
    base='A'면 A 기준 이웃에서 선택, 'B'면 B 기준.
    """
    anchor = A if base == "A" else B
    cand = kv.most_similar(anchor, topn=topk)
    cand = [w for w, sc in cand if w not in (A, B) and min_sim <= sc <= max_sim]
    if not cand:  # 실패 시 반대 축으로 재시도
        anchor = B if base == "A" else A
        cand = kv.most_similar(anchor, topn=topk)
        cand = [w for w, sc in cand if w not in (A, B) and min_sim <= sc <= max_sim]
    if not cand:
        return None
    return random.choice(cand)

def get_analogy_topk(kv, A, B, C, topn=5):
    """b - a + c와 가장 가까운 상위 후보들."""
    return kv.most_similar(positive=[B, C], negative=[A], topn=topn)

def cosine(a, b):
    na, nb = norm(a), norm(b)
    if na == 0 or nb == 0: return 0.0
    return float(np.dot(a, b) / (na * nb))

def score_user_answer(kv, A, B, C, user_word):
    """사용자 답을 아날로지 벡터(b-a+c)와 직접 코사인으로 채점."""
    if user_word not in kv.key_to_index:
        return None
    vec = kv[B] - kv[A] + kv[C]
    return cosine(kv[user_word], vec)

def play_game(kv, ab_th=0.75):
    # 1) 유사한 A,B 고르기
    A, B, simAB = get_word_pair(kv, threshold=ab_th)

    # 2) A(또는 B)와도 유사한 C 고르기
    C = pick_C_similar(kv, A, B, base=random.choice(["A","B"]))
    if C is None:
        # 후보 못 찾았으면 다시 시작
        return play_game(kv, ab_th)

    # 3) 문제 출력
    print(f"관계 [ {A} : {B} = {C} : ? ]  (A~B 유사도 {simAB:.2f})")
    user_a = input("알맞은 답을 입력하세요: ").strip()

    # 4) 모델 예측(상위 N개 후보 중 하나를 랜덤 정답으로 선택)
    top = get_analogy_topk(kv, A, B, C, topn=5)

    # top 리스트가 비어있지 않다면 랜덤으로 하나 선택
    predicted_D, pred_score = random.choice(top)

    # 허용 정답 후보 (상위 N개 전부)
    allow = {w for w, _ in top}

    print(f"모델이 랜덤으로 선택한 정답: {predicted_D} (유사도 {pred_score:.3f})")
    print("허용 정답 후보:", ", ".join(allow))

    # 5) 채점
    if user_a in kv.key_to_index:
        # (a) 아날로지 벡터와의 직접 유사도
        ana_sim = score_user_answer(kv, A, B, C, user_a)
        # (b) 모델 1위와의 유사도(참고용)
        ref_sim = kv.similarity(user_a, predicted_D)
        print(f"- 아날로지 벡터와 유사도: {ana_sim:.3f}")
        print(f"- 모델1위({predicted_D})와 유사도: {ref_sim:.3f}")
    else:
        print("입력한 단어는 모델 어휘에 없습니다.")

    # 최종 판정
    if user_a in allow:
        print("정답!")
    else:
        print("오답!")

In [102]:
while True:
        play_game(load_model)
        again = input("\n다시 하시겠습니까? (y/n): ").strip()
        if again != "ㅇ":
            print("게임을 종료합니다.")
            break

관계 [ 공이 : 최고다 = 증 : ? ]  (A~B 유사도 0.76)
모델이 랜덤으로 선택한 정답: 꿀잼 (유사도 0.821)
허용 정답 후보: 꿀잼, 포스, 라임, 오래오래, 전성기
입력한 단어는 모델 어휘에 없습니다.
오답!
관계 [ 코미디 : 오버 = 라미란 : ? ]  (A~B 유사도 0.76)
모델이 랜덤으로 선택한 정답: 수지 (유사도 0.774)
허용 정답 후보: 워치, 케미, 무한도전, 신의한수, 수지
- 아날로지 벡터와 유사도: 0.617
- 모델1위(수지)와 유사도: 0.828
오답!
관계 [ 저승사자 : 예나 = 정장 : ? ]  (A~B 유사도 0.78)
모델이 랜덤으로 선택한 정답: 니놈 (유사도 0.795)
허용 정답 후보: 계정, 미나, 국뽕, 베트남, 니놈
입력한 단어는 모델 어휘에 없습니다.
오답!
게임을 종료합니다.


---